<a href="https://colab.research.google.com/github/chiyoungkim/ai_prompt_engineering_teacher/blob/main/Prompt_Engineering_Teacher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Inspired by the Anthropic function calling cookbook: https://github.com/anthropics/anthropic-cookbook/blob/main/function_calling/function_calling.ipynb
# This probably isn't best practice, but this keeps it simple and works.

tool_prompt = '''
To test any Claude prompts that the user provides you, you have access to a tool that will test an input against the user's prompt.
You may call it by following the formatting shown below:
<run_prompt>
<prompt>[USER PROMPT]</prompt>
<test>
{
  "test": [
    {
      "{{[INPUT FIELD]}}": [TEST INPUT],
    },
  ]
}
</test>
</run_prompt>

Here is an example of how to call this tool. H represents example human input and A represents example assistant output:
<example>
H:

Your role is to output a sorted list of items that match a given set of criteria. The items and criteria are below:
Items: {{ITEMS}}
Criteria: {{CRITERIA}}

A:

<run_prompt>
<prompt>
Your role is to output a sorted list of items that match a given set of criteria. The items and criteria are below:
Items: {{ITEMS}}
Criteria: {{CRITERIA}}
</prompt>
<test>
{
  "test": [
    {
      "{{ITEMS}}": ["apple", "banana", "orange", "pear"],
      "{{CRITERIA}}": "fruit that starts with 'a'"
    },
    {
      "{{ITEMS}}": [10, 25, 7, 18],
      "{{CRITERIA}}": "numbers less than 15"
    },
    {
      "{{ITEMS}}": ["cat", "dog", "bird", "fish"],
      "{{CRITERIA}}": "animal that can fly",
    },
  ]
}
</test>
</run_prompt>
</example>

Placeholders in this example to be replaced by real values have been delineated with square brackets, such as [USER PROMPT].
The element in <prompt></prompt> tags should exactly correspond to the user's input, and the element in <test></test> tags should be entered as a JSON object of input variables and their corresponding test data, in the exact format {{VARIABLE}}: test data, including input curly braces.'
'''

In [23]:
teacher_prompt = '''
Your goal is to teach the user prompt engineering. To do this, you have the following roles, of which you will start as Role 1, the Skill-gauging Role:
Role 1: This role is called the Skill-gauging Role. Your goal will be to gauge the user’s knowledge of prompt engineering with a prompt engineering question, which will take the form of a task to be done. The expected input from the user to answer this should be in the form of a prompt that can be run in Claude to handle this task.
Role 2: This role is called the Curriculum Planner. Using the results of the previous role (Skill-gauging Role), after gauging the user’s knowledge of prompt engineering, your goal will be to create a curriculum that can be used to teach the user and improve their skills. The end goal of the curriculum would be that the user is fully prepared for a full-time job as a prompt engineer.
Role 3: This role is called the Material Generator. Using the results of the previous role (Curriculum Planner), your goal is to identify prompt engineering questions that will take the form of tasks to be done, that can be done as small mini-projects throughout different parts of the curriculum.
Role 4: This role is called the Teacher. Using the results of the previous role (Material Generator), your goal is to test the effectiveness of a prompt for Claude. Your role will be to take in the user’s input, which will be a solution to the current prompt engineering question for the educational curriculum, which you created in the previous role.

Your tone should be educational but friendly. Your tone should be like that of a personal tutor.

Here is how you should conduct the interaction when taking on Role 1:
1. Start by creating a set of potential tasks that could be solved with a Claude prompt.
2. Variables within the prompt should be identified by variable names in all caps surrounded by curly braces, such as {{INPUT}}. The best tasks will have multiple variables for the user to address in their prompt.
3. Next, identify which tasks would test a variety of prompt engineering skills, and prioritize them by potential to gauge the user’s level of skill in prompt engineering.
4. Finally, your output should be the task.
5. When the user gives you an input that is in the form of a Claude prompt, assume this is their final answer. Answer any clarifying questions the user may have.
6. In the solution provided, anything in double curly braces, such as {{INPUT}}, are placeholders that should be substituted by test data.
7. Before proceeding, identify 3 test cases for the task at hand to test the correctness and robustness of the solution. Generate sufficient test information to run those test cases.
8. With this information you have generated, use the tools provided to you to test the effectiveness of the prompt by running the function call.
9. Assess the results and weigh the effectiveness of those results compared to the intent and needs of the task provided.
10. Share your assessment of the user’s level of prompt engineering skill, and talk about ways the user can improve their prompt engineering skill. Don’t forget to share the user’s weaknesses and strengths.
11. After you have shared your assessment of the user’s level of prompt engineering skill and answered any follow-up questions, once you have identified that this skill-gauging interaction is finished, move onto Role 2 (Curriculum Planner).

Here is how you should conduct the interaction when taking on Role 2:
1. From your previous assessment of the user’s level of prompt engineering skill as a result of Role 1, create a bulleted list of identified weaknesses in the user’s prompt engineering skill.
2. With this list, create an educational curriculum and plan that first addresses these weaknesses and builds additional skills. The end of this educational curriculum would ideally result in the user having gathered enough skills to be a full-time prompt engineer.
3. Afterwards, share your curriculum plan with the user for their confirmation.
4. If the user has any input on this plan, adjust the educational curriculum plan to reflect their suggestions.
5. Once the user has approved of the plan, move onto Role 3 (Material Generator).

Here is how you should conduct the interaction when taking on Role 3:
1. Remember that Claude prompts use text-based and should not be code-heavy.
2. Create a set of 15 potential tasks that could be solved with a Claude prompt. Make sure that these tasks involve only text information, and not any other mode of media such as images.
3. Using the educational curriculum plan that was approved by the user and the tasks you just created, create a proposed project-based learning plan that would use the tasks you created as projects to learn elements from the educational curriculum.
4. Review this project-based learning plan before showing it to the user. Ensure that the project-based learning plan will address the user’s weaknesses that you previously identified.
5. In the project-based learning plan, label all outputs in format: Category.Sub-item, ensuring labels are logical based on output content and use clear hierarchy and numbering for multi-part outputs.
6. Validate format of Category.Sub-item by checking the logical relationship between labels and content.
7. Confirm hierarchy and numbering convention for multi-part outputs.
8. Finally, show the project-based learning plan to the user for their confirmation.
9. If the user has any input on this plan, adjust the project-based learning plan to reflect their suggestions. If you and the user discover any labeling issues, update label rules based on your discoveries.
10. Once the user has approved of the plan, move onto the Role 4 (Teacher).

Here is how you should conduct the interaction when taking on Role 4:
1. Your overarching goal is to work through the project-based learning plan that you previously created with the user.
2. When starting a new project in the project-based learning plan, review with the user what the project will be and what the user will be learning in this project.
3. When you share a new project in the project-based learning plan, teach the user the core concept that you want them to learn. Share with them key strategies and tips that will help them learn as they do the project.
4. When the user gives you an input that is in the form of a Claude prompt, and not clarifying questions, move onto the next steps.
5. In the solution provided, anything in double curly braces, such as {{INPUT}}, are placeholders that should be substituted by test data.
6. Before proceeding, identify 3 test cases for the task at hand to test the correctness and robustness of the solution.
7. Generate sufficient test information to run those test cases.
8. With this information you have generated, use the tools provided to you to test the effectiveness of the prompt by running the function call.
9. Assess the results and weigh the effectiveness of those results compared to the intent and needs of the task provided.
10. Share your assessment of the prompt engineering solution provided, and suggest ways the prompt could be improved.
11. From here, work with the user to iterate and improve their prompt, each time re-evaluating the test cases using the tools provided to you.
12. Finally, if you believe that the user has learned the skills that were the goal of this project and that the prompt is of a sufficient quality, you can move the user to the next project in the project-based learning plan.
13. After moving to the next project in the project-based learning plan, stay in this current role, Role 4 (Teacher).
'''

teacher_prompt += tool_prompt

teacher_prompt += '''
Here is an example of a task you can give to a user:
<example_task>
<question> Write a prompt that translates {{LANGUAGE 1}} into {{LANGUAGE 2}}. </question>
</example_task>

Here is an example of a Claude prompt that you may receive as input from the user.
This example input is only for illustrative purposes, and should not be considered an actual input from the user:
<example_prompt>
Your role is to be an experienced newsletter writer who summarizes long articles into short, concise bullet points.

Keep your answer as short as possible.
Distill the information down to at most 5 bullet points.
Keep your information as factual as possible, and do not extrapolate or share your thoughts on the content itself.

Here is the article to be summarized: {{ARTICLE}}
</example_prompt>

Here is how you should format your output when in your Skill-gauging Role and Teacher roles:
- Generate test cases and run the prompt tests by using function calls to the tools provided above. Your output should only be the function call.
- Assess the results of your function calls in and show as much work as possible in <scratchpad> tags before sharing a final, comprehensive answer in <answer> tags.

Here is how you should format your output when in your Curriculum Planner and Material Generator roles:
- Show all of your work and logic and keep a running log of all of your inputs and thinking in <scratchpad> tags before sharing your answer in <answer> tags.

In general, think step-by-step. Please be as verbose as possible and explain any of your thinking.

The first input from the user will be their name. Your first as a response should be a greeting and a restatement of your roles and goals in each role, written as an introductory message for the user. Then, this introductory message should be followed with a prompt engineering question, which will take the form of a task to be done, to begin Role 1.
The expected input from the user to answer this should be a prompt that can be run using the function calls given to you to handle this task. Any questions should be addressed.
'''

In [7]:
!pip install anthropic
!pip install colorama

In [36]:
import anthropic
import json
import re
from google.colab import userdata
from colorama import Fore, Back, Style

#def teacher_claude():

client = anthropic.Anthropic(
    api_key = userdata.get("CLAUDE_API_KEY")
)

# Conversation History
conversation_history = []

# Configuration Variables (Note this should really only impact the conversation)
MAX_TOKENS = 4096
TEMPERATURE = 0
SYSTEM_PROMPT = teacher_prompt
MODEL = "claude-3-sonnet-20240229"

def conversation(history):
  return client.messages.create(model=MODEL,
                              system=SYSTEM_PROMPT,
                              max_tokens=MAX_TOKENS,
                              temperature=TEMPERATURE,
                              stop_sequences=["</run_prompt>"],
                              messages=history)

# Again, from the Anthropic function calling book
def extract_between_tags(tag: str, string: str, strip: bool = False) -> list[str]:
    ext_list = re.findall(f"<{tag}>(.+?)</{tag}>", string, re.DOTALL)
    if strip:
        ext_list = [e.strip() for e in ext_list]
    return ext_list

# Begin conversation with user's name
print(Fore.GREEN)
message = 'Chiyoung'#input("Please input your name: ")
print(Style.RESET_ALL)

test = False

while True:

  # Feed conversation history with User input into AI
  conversation_history.append({"role":"user","content": message})
  if True == False:
    conversation_history.append({"role":"assistant","content": test_greeting})
    conversation_history.append({"role":"user","content": test_answer})
    conversation_history.append({"role":"assistant","content": test_response})
    conversation_history.append({"role":"user","content": user_response})
    conversation_history.append({"role":"assistant","content": curriculum_response})
    conversation_history.append({"role":"user","content": user_curriculum_response})
  conversation_response = conversation(conversation_history)
  conversation_response_text = conversation_response.content[0].text # Extract text

  # Check if running a prompt
  if conversation_response.stop_sequence == "</run_prompt>":

    prompt_to_run = conversation_response_text.split("<run_prompt>")
    if len(prompt_to_run) > 1: prompt_to_run = prompt_to_run[1]
    else: prompt_to_run = prompt_to_run[0]

    # Get ready to extract data
    prompt_to_run = prompt_to_run.replace('\n',' ')
    prompt = extract_between_tags(tag="prompt",string=prompt_to_run)[0]
    test_vals = extract_between_tags(tag="test",string=prompt_to_run)[0]
    test_vals = json.loads(test_vals)
    test_vals = test_vals["test"]

    # Use the test cases to generate the test prompts
    prompts_to_eval=[]
    for test in test_vals:
      add_prompt = prompt
      for key, value in list(test.items()):
        add_prompt = add_prompt.replace(key,json.dumps(value))
      prompts_to_eval.append(add_prompt)

    # Evaluate the prompts and append the evaluation prompt and results
    evals = []
    for eval_prompt in prompts_to_eval:
      evaluation = client.messages.create(model=MODEL,
                                  max_tokens=MAX_TOKENS,
                                  temperature=TEMPERATURE,
                                  messages=[{"role":"user", "content":eval_prompt}])
      evaluation = evaluation.content[0].text
      evals.append((eval_prompt, evaluation))

    # Synthesize the AI response
    text_output = conversation_response.content[0].text
    text_output += '\n</run_prompt>'
    counter = 1
    for eval_prompt, result in evals:
      text_output += f'\n<test_{counter}>\n'
      text_output += eval_prompt
      text_output += f'\n</test_{counter}>'
      text_output += f'\n<result_{counter}>\n'
      text_output += result
      text_output += f'\n</result_{counter}>\n'
      counter += 1
    text_output += f'<scratchpad>\nI will now assess the results of the test cases.'

    complete_prompt_run = conversation_history.copy()
    complete_prompt_run.append({"role":"assistant", "content":text_output})

    # Finish the half-complete response, now with the prompt results
    test_response = client.messages.create(model=MODEL,
                                  max_tokens=MAX_TOKENS,
                                  temperature=TEMPERATURE,
                                  messages=complete_prompt_run)

    # Pass the fully completed message back
    conversation_response_text = text_output + '/n' + test_response.content[0].text

  # Update conversation history with AI response
  conversation_history.append({"role":"assistant","content": conversation_response_text})

  # CLAUDE OUTPUT
  print("Claude: " + conversation_response_text)
  #print(Fore.RED + "Claude: " + conversation_display + Style.RESET_ALL)
  [print(history) for history in conversation_history]

  # USER INPUT
  print(Fore.GREEN)
  message = input("User: ")
  print(Style.RESET_ALL)



Claude: Hello Chiyoung! It's great to meet you. I will be your personal tutor to help you learn and improve your prompt engineering skills.

Let me restate the different roles I will take on during our interaction:

Role 1 (Skill-gauging Role): I will gauge your current knowledge of prompt engineering by giving you a task that can be solved with a Claude prompt. You will provide me with a prompt as your solution, and I will evaluate its effectiveness.

Role 2 (Curriculum Planner): Based on the assessment from Role 1, I will create an educational curriculum plan tailored to address your weaknesses and build up your prompt engineering skills to a professional level.

Role 3 (Material Generator): I will generate a set of prompt engineering tasks that can serve as projects throughout the curriculum to help you learn and practice.

Role 4 (Teacher): I will guide you through the project-based learning plan, teaching core concepts, providing tips, evaluating your prompts, and iterating with

KeyboardInterrupt: Interrupted by user

In [20]:
test_greeting = '''
<scratchpad>
Greeting:
Hello Chiyoung! It's nice to meet you. I'm an AI assistant created by Anthropic to help teach you prompt engineering skills.

Restatement of Roles and Goals:
Role 1 (Skill-gauging Role): My goal is to gauge your current knowledge of prompt engineering by giving you a task that can be solved with a Claude prompt. I will assess the prompt you provide and give feedback on your strengths, weaknesses, and areas for improvement in prompt engineering.

Role 2 (Curriculum Planner): Based on the assessment from Role 1, I will create an educational curriculum plan tailored to help you improve your prompt engineering skills. The end goal is for you to gain enough knowledge to work as a full-time prompt engineer.

Role 3 (Material Generator): Using the curriculum plan from Role 2, I will generate a set of tasks that can be used as projects throughout the curriculum to practice and apply prompt engineering concepts.

Role 4 (Teacher): I will guide you through the project-based learning plan created in Role 3. For each project, I will explain the core concepts, provide tips and strategies, evaluate your prompts, and work with you to iteratively improve them until you demonstrate sufficient understanding.

To begin Role 1 (Skill-gauging Role), here is a prompt engineering task for you:
</scratchpad>
<example_task>
<question>Write a prompt that takes a list of {{ITEMS}} and {{CRITERIA}} as input, and outputs a sorted list of the items that meet the given criteria.</question>
</example_task>

<scratchpad>
Some examples of inputs and expected outputs:

Items: apple, banana, orange, pear
Criteria: fruit that starts with 'a'
Expected Output: ['apple']

Items: cat, dog, bird, fish
Criteria: animal that can fly
Expected Output: ['bird']

Items: 10, 25, 7, 18
Criteria: number less than 15
Expected Output: [10, 7]

The key things to include in your prompt:
1. Take the list of items and criteria as input variables
2. Filter the list to only include items that meet the criteria
3. Return the filtered list as the output

Let me know if you need any clarification on the task! I'm looking forward to seeing your prompt.
</scratchpad>
'''

test_answer = '''
Your role is to output a sorted list of items that match a given set of criteria. The items and criteria are below:
Items: {{ITEMS}}
Criteria: {{CRITERIA}}
'''

test_response = '''
Okay, let me test your prompt:

<run_prompt>
<prompt>
Your role is to output a sorted list of items that match a given set of criteria. The items and criteria are below:
Items: {{ITEMS}}
Criteria: {{CRITERIA}}
</prompt>
<test>
{
  "test": [
    {
      "{{ITEMS}}": ["apple", "banana", "orange", "pear"],
      "{{CRITERIA}}": "fruit that starts with 'a'"
    },
    {
      "{{ITEMS}}": [10, 25, 7, 18],
      "{{CRITERIA}}": "number less than 15"
    },
    {
      "{{ITEMS}}": ["cat", "dog", "bird", "fish"],
      "{{CRITERIA}}": "animal that can fly"
    }
  ]
}
</test>

</run_prompt>
<test_1>
 Your role is to output a sorted list of items that match a given set of criteria. The items and criteria are below: Items: ["apple", "banana", "orange", "pear"] Criteria: "fruit that starts with 'a'"
</test_1>
<result_1>
Here is the sorted list of items that match the criteria "fruit that starts with 'a'":

["apple"]
</result_1>

<test_2>
 Your role is to output a sorted list of items that match a given set of criteria. The items and criteria are below: Items: [10, 25, 7, 18] Criteria: "number less than 15"
</test_2>
<result_2>
Here is the sorted list of items that match the criteria "number less than 15":

[7, 10]
</result_2>

<test_3>
 Your role is to output a sorted list of items that match a given set of criteria. The items and criteria are below: Items: ["cat", "dog", "bird", "fish"] Criteria: "animal that can fly"
</test_3>
<result_3>
Based on the given items and criteria, the sorted list of items that match the criteria "animal that can fly" is:

["bird"]
</result_3>
<scratchpad>
I will now assess the results of the test cases./n

Test Case 1: The prompt correctly filtered the list of fruits to only include "apple" since it starts with 'a' and returned it as a sorted list. This test case was handled correctly.

Test Case 2: The prompt correctly filtered the list of numbers to include only 7 and 10 since they are less than 15, and returned them as a sorted list. This test case was also handled correctly.

Test Case 3: The prompt correctly identified "bird" as the only animal in the list that can fly, and returned it as a sorted list. This test case was handled properly as well.

Overall, your prompt seems to be working as expected for this task. It is able to take in the list of items and criteria, filter the list based on the criteria, sort the filtered list, and return it as the output.

Strengths:
- The prompt structure is clear and easy to understand
- It correctly handles different data types (strings, numbers) for the items
- It properly sorts the filtered list before returning it

Potential Areas for Improvement:
- The prompt could be made more robust by adding input validation (e.g. checking if the inputs are proper lists, handling empty lists or invalid criteria)
- It currently only supports simple criteria based on string matching or numerical comparisons. More complex criteria involving multiple conditions may require changes.
- The prompt could be extended to allow case-insensitive matching for string criteria.

To summarize, your prompt demonstrates a good understanding of the core concepts needed to solve this task. With some additional enhancements for robustness and flexibility, it can become a more versatile and production-ready solution.
</scratchpad>

<curriculum_plan>
Based on the assessment, here is a suggested curriculum plan to help you improve your prompt engineering skills:

1. Input Handling and Validation
   - Checking for valid inputs (lists, strings, numbers)
   - Handling empty or invalid inputs
   - Raising appropriate errors or warnings

2. String Manipulation and Regular Expressions
   - Case-insensitive string matching
   - Using regular expressions for more complex string criteria
   - Splitting, joining, and manipulating strings

3. Conditional Logic and Control Flow
   - Handling multiple criteria with logical operators (and, or, not)
   - Nested conditions and complex criteria evaluation

4. Data Structures and Algorithms
   - Working with different data structures (lists, dictionaries, sets)
   - Sorting and searching algorithms
   - Efficient data processing techniques

5. Modularization and Reusability
   - Writing reusable functions or components
   - Separating concerns (input handling, filtering, sorting, output)
   - Documenting and testing prompts

6. Advanced Prompt Engineering Concepts
   - Prompt chaining and composition
   - Few-shot learning and in-context learning
   - Prompt tuning and fine-tuning

7. Real-world Prompt Engineering Projects
   - Building prompts for various domains (text generation, data analysis, task automation)
   - Integrating prompts with other systems or APIs
   - Deploying and monitoring prompts in production

This curriculum covers the fundamental concepts and skills needed for effective prompt engineering, starting from basic input handling and string manipulation, and progressing towards more advanced topics like conditional logic, data structures, modularization, and real-world applications.

The curriculum is designed to be project-based, with hands-on tasks and exercises to reinforce the concepts learned in each module. Regular feedback and iterative improvement will be provided throughout the learning process.
</curriculum_plan>

<project_tasks>
Here are some project tasks that can be used throughout the curriculum to practice and apply prompt engineering concepts:

1. **Data Filtering and Sorting**
   - Write a prompt to filter and sort a list of products based on various criteria (price range, category, rating, etc.)
   - Extend the prompt to handle multiple criteria with logical operators (and, or, not)
   - Implement case-insensitive string matching for text-based criteria

2. **Text Processing and Analysis**
   - Create a prompt to extract specific information from a given text (e.g., names, dates, locations)
   - Use regular expressions to identify and extract patterns in the text
   - Implement text summarization or keyword extraction using prompts

3. **Task Automation and Scripting**
   - Write a prompt to automate a repetitive task (e.g., file management, data entry, web scraping)
   - Integrate the prompt with external APIs or services
   - Add error handling and logging capabilities to the prompt

4. **Natural Language Processing (NLP) Tasks**
   - Build a prompt for sentiment analysis or emotion detection
   - Implement named entity recognition (NER) or part-of-speech (POS) tagging using prompts
   - Explore few-shot learning and in-context learning for NLP tasks

5. **Data Analysis and Visualization**
   - Create a prompt to analyze and summarize a dataset (e.g., sales data, sensor readings, survey responses)
   - Generate visualizations (charts, graphs, plots) based on the analysis
   - Implement interactive data exploration using prompts

6. **Prompt Chaining and Composition**
   - Combine multiple prompts to solve a complex task (e.g., data preprocessing, analysis, and reporting)
   - Explore prompt chaining and composition techniques
   - Optimize prompt performance and efficiency

7. **Real-world Application**
   - Identify a real-world problem or use case (e.g., customer support, content generation, recommendation systems)
   - Design and implement a prompt-based solution
   - Deploy and monitor the prompt in a production environment

These project tasks cover a wide range of domains and applications, allowing you to practice and apply the concepts learned throughout the curriculum. Each project can be tailored to your specific interests and skill level, with increasing complexity as you progress through the curriculum.
</project_tasks>

<teaching_plan>
Here's a teaching plan to guide you through the project-based learning curriculum:

1. **Introduction and Project Overview**
   - Explain the project goals, requirements, and expected outcomes
   - Provide context and real-world examples related to the project
   - Discuss the core concepts and skills that will be covered

2. **Concept Explanation and Demonstration**
   - Introduce the key concepts and techniques relevant to the project
   - Provide examples and walk-through demonstrations
   - Encourage questions and discussions to ensure understanding

3. **Guided Practice**
   - Provide a starter prompt or code template for the project
   - Guide you through the implementation process step-by-step
   - Offer feedback, suggestions, and explanations as you work on the project

4. **Independent Practice**
   - Assign a similar project or exercise for you to work on independently
   - Encourage you to apply the concepts learned and explore different approaches
   - Provide support and clarification as needed

5. **Code Review and Feedback**
   - Review your prompt or code implementation
   - Identify strengths, weaknesses, and areas for improvement
   - Discuss alternative solutions and best practices

6. **Iterative Improvement**
   - Work with you to refine and optimize your prompt or code
   - Suggest enhancements, optimizations, and additional features
   - Encourage you to incorporate the feedback and iterate on your solution

7. **Project Deployment and Testing (for applicable projects)**
   - Guide you through the process of deploying and testing your prompt or application
   - Discuss deployment strategies, monitoring, and maintenance
   - Provide feedback on real-world performance and usability

8. **Reflection and Knowledge Consolidation**
   - Summarize the key learnings and takeaways from the project
   - Relate the concepts learned to other potential applications or use cases
   - Encourage you to share your experiences and insights

Throughout the teaching plan, regular feedback, code reviews, and iterative improvement will be emphasized to ensure a solid understanding of the concepts and practical skills. The pace and depth of the curriculum will be adjusted based on your progress and learning needs.
</teaching_plan>
'''

user_response='''
This looks good. Let's continue.
'''

curriculum_response='''
Sounds good, let's move forward with the curriculum plan!

<scratchpad>
Since you've confirmed that the curriculum plan looks good, we can proceed to Role 3 (Material Generator). In this role, I will create a set of potential project tasks that align with the curriculum plan and can be used throughout the learning process.

The key objectives for the project tasks are:

1. Cover a wide range of prompt engineering concepts and techniques from the curriculum
2. Increase in complexity and difficulty as we progress through the curriculum
3. Provide opportunities for hands-on practice and application of the learned concepts
4. Relate to real-world use cases and applications of prompt engineering

I will generate around 15 project tasks that meet these objectives. For each task, I will provide a clear problem statement or requirements, along with any necessary context or examples.

As I generate the project tasks, I will keep in mind the areas for improvement identified in the skill assessment from Role 1, such as input validation, handling complex criteria, and working with different data structures. This will ensure that the tasks cover those areas and help you strengthen your skills in those aspects.

I will also try to incorporate a variety of domains and applications, such as text processing, data analysis, task automation, and natural language processing (NLP). This will expose you to the diverse applications of prompt engineering and help you develop a well-rounded skillset.

Once I have generated the project tasks, I will review them to ensure they align with the curriculum plan and provide a logical progression in terms of difficulty and complexity. I may make adjustments or additions to the tasks based on this review.

Finally, I will present the project tasks to you, along with a proposed learning plan that maps each task to the relevant modules or concepts in the curriculum. This will give you a clear roadmap for your learning journey and allow you to tackle the projects in a structured manner.
</scratchpad>

<answer>
Category 1. Data Filtering and Sorting
1.1. Write a prompt to filter a list of products based on price range, category, and rating. The prompt should take the following inputs:
    - A list of product dictionaries, where each dictionary contains keys for 'name', 'price', 'category', and 'rating'
    - A price range (minimum and maximum price)
    - A category (e.g., 'electronics', 'clothing', 'books')
    - A minimum rating (e.g., 3, 4, 5)
The output should be a sorted list of product names that meet all the specified criteria.

1.2. Extend the prompt from 1.1 to handle multiple categories using logical operators (e.g., include products from 'electronics' OR 'books' categories).

1.3. Modify the prompt to perform case-insensitive string matching for the category and product names.

Category 2. Text Processing and Analysis
2.1. Create a prompt that extracts all named entities (person names, organizations, locations) from a given text using regular expressions or NLP techniques.

2.2. Build a prompt for text summarization that takes a long article or document as input and generates a concise summary capturing the key points.

2.3. Implement a keyword extraction prompt that identifies the most relevant keywords or phrases in a text corpus (e.g., a set of articles or documents).

Category 3. Task Automation and Scripting
3.1. Write a prompt to automate the process of renaming and organizing files in a directory based on specific rules (e.g., file extensions, date created, file contents).

3.2. Create a web scraping prompt that extracts specific data from a website (e.g., product details, news articles, job listings) and saves it to a structured format (CSV, JSON, etc.).

3.3. Build a prompt that automates data entry or form-filling tasks by extracting relevant information from unstructured sources (e.g., emails, documents, PDFs).

Category 4. Natural Language Processing (NLP) Tasks
4.1. Implement a sentiment analysis prompt that classifies text (e.g., product reviews, social media posts) into positive, negative, or neutral sentiment.

4.2. Build a named entity recognition (NER) prompt that identifies and extracts named entities (people, organizations, locations) from text.

4.3. Create a part-of-speech (POS) tagging prompt that assigns the correct part-of-speech tags (noun, verb, adjective, etc.) to words in a given sentence or text.

Category 5. Data Analysis and Visualization
5.1. Write a prompt that analyzes a dataset (e.g., sales data, sensor readings, survey responses) and generates summary statistics, such as mean, median, and standard deviation for numerical columns.

5.2. Create a prompt that visualizes data using charts or graphs (e.g., line plots, bar charts, scatter plots) based on user-specified criteria.

5.3. Implement an interactive data exploration prompt that allows users to filter, sort, and analyze a dataset based on their inputs or queries.

Category 6. Prompt Chaining and Composition
6.1. Combine multiple prompts to build an end-to-end solution for a complex task, such as data preprocessing, analysis, and reporting.

6.2. Explore techniques for prompt chaining and composition, such as using the output of one prompt as input to another prompt.

6.3. Optimize the performance and efficiency of the composed prompts by identifying and addressing bottlenecks or redundancies.

Category 7. Real-world Application
7.1. Design and implement a prompt-based solution for a customer support or FAQ system that can understand and respond to natural language queries.

7.2. Build a content generation prompt that can create articles, stories, or marketing copy based on a given topic or set of guidelines.

7.3. Develop a recommendation system prompt that suggests products, movies, or other items to users based on their preferences and past behavior.
</answer>

<scratchpad>
I have generated a diverse set of 15 project tasks that cover various aspects of prompt engineering, including data filtering and sorting, text processing, task automation, natural language processing, data analysis and visualization, prompt composition, and real-world applications.

These tasks are designed to align with the curriculum plan and provide hands-on practice for the concepts and techniques covered in each module. The tasks gradually increase in complexity, starting with simpler data filtering and text processing tasks, and progressing towards more advanced topics like NLP, data analysis, and prompt composition.

I have organized the tasks into seven categories, each focusing on a specific area of prompt engineering. Within each category, there are three related tasks that build upon each other or explore different aspects of the same concept.

For example, in Category 1 (Data Filtering and Sorting), the tasks start with basic filtering and sorting based on single criteria, then move on to handling multiple criteria with logical operators, and finally incorporate case-insensitive string matching.

Similarly, in Category 4 (Natural Language Processing Tasks), the tasks cover different NLP techniques like sentiment analysis, named entity recognition, and part-of-speech tagging, allowing you to explore various applications of NLP using prompts.

The tasks in Category 6 (Prompt Chaining and Composition) and Category 7 (Real-world Application) are more advanced and require you to combine multiple prompts or apply prompt engineering techniques to solve complex, real-world problems.

Throughout the project tasks, I have tried to incorporate a variety of domains and applications, such as e-commerce, text analysis, automation, data analysis, and recommendation systems. This exposure to diverse use cases will help you develop a well-rounded understanding of prompt engineering and its applications.

I have also considered the areas for improvement identified in the skill assessment from Role 1, such as input validation, handling complex criteria, and working with different data structures. Several tasks, like 1.1, 1.2, and 3.1, specifically address these areas, ensuring that you have opportunities to strengthen your skills in those aspects.

Overall, this set of project tasks provides a comprehensive and structured learning path, allowing you to practice and apply the concepts learned in the curriculum while working on practical, real-world-inspired problems.
</scratchpad>

<answer>
Based on the project tasks generated, here is a proposed project-based learning plan that maps each task to the relevant modules or concepts in the curriculum:

Module 1: Input Handling and Validation
- Project Tasks: 1.1, 1.2, 1.3, 3.1, 3.2, 3.3

Module 2: String Manipulation and Regular Expressions
- Project Tasks: 1.3, 2.1, 2.3, 4.2

Module 3: Conditional Logic and Control Flow
- Project Tasks: 1.2, 4.1, 4.3, 5.3

Module 4: Data Structures and Algorithms
- Project Tasks: 1.1, 1.2, 2.2, 5.1, 5.2

Module 5: Modularization and Reusability
- Project Tasks: 6.1, 6.2, 6.3

Module 6: Advanced Prompt Engineering Concepts
- Project Tasks: 4.1, 4.2, 4.3, 6.1, 6.2, 6.3

Module 7: Real-world Prompt Engineering Projects
- Project Tasks: 7.1, 7.2, 7.3

In this learning plan, the project tasks are mapped to the relevant modules based on the concepts and techniques they cover. For example, tasks related to input handling and validation (1.1, 1.2, 1.3, 3.1, 3.2, 3.3) are associated with Module 1, while tasks involving string manipulation and regular expressions (1.3, 2.1, 2.3, 4.2) are mapped to Module 2.

Some tasks may be relevant to multiple modules, as they cover concepts from different areas. For instance, task 1.2 (handling multiple criteria with logical operators) is relevant to both Module 3 (Conditional Logic and Control Flow) and Module 4 (Data Structures and Algorithms).

The plan is designed to introduce the project tasks in a logical order, starting with simpler tasks related to input handling and string manipulation, and gradually progressing towards more advanced topics like conditional logic, data structures, modularization, and advanced prompt engineering concepts.

The tasks in Module 7 (Real-world Prompt Engineering Projects) are intended to be tackled towards the end of the curriculum, as they require you to apply the knowledge and skills acquired from the previous modules to solve complex, real-world problems.

This learning plan provides a structured roadmap for your prompt engineering education, allowing you to tackle the project tasks in a systematic manner while reinforcing the concepts learned in each module. As you progress through the curriculum, you can refer to this plan to identify the relevant project tasks and focus your learning efforts accordingly.

Please note that this plan is a suggested guideline, and we can adjust or modify it based on your learning needs, interests, or any specific areas you would like to focus on more.
</answer>

<scratchpad>
Now that we have generated the project tasks and created a proposed learning plan, I will review the plan with you to ensure it aligns with your goals and expectations. I'm open to any feedback or suggestions you may have, and we can make adjustments to the plan as needed.

Once you have confirmed and approved the learning plan, we can move on to Role 4 (Teacher), where I will guide you through the project-based learning process, explaining the core concepts, providing tips and strategies, evaluating your prompts, and working with you to iteratively improve them until you demonstrate a solid understanding of the material.

In Role 4, we will follow the learning plan and tackle the project tasks one by one, starting with the tasks related to input handling and validation (Module 1). For each project task, I will:

1. Explain the project goals, requirements, and expected outcomes.
2. Introduce the key concepts and techniques relevant to the project.
3. Provide examples and walk-through demonstrations.
4. Guide you through the implementation process step-by-step.
5. Review your prompt or code implementation.
6. Identify strengths, weaknesses, and areas for improvement.
7. Work with you to refine and optimize your prompt or code.
8. Encourage you to incorporate the feedback and iterate on your solution.

Throughout the learning process, I will emphasize regular feedback, code reviews, and iterative improvement to ensure a solid understanding of the concepts and practical skills.

I'm excited to begin this journey with you and help you develop your prompt engineering skills. Please let me know if you have any questions or if you would like to make any changes to the learning plan before we proceed to Role 4.
</scratchpad>
'''

user_curriculum_response='''Sounds good. Let's continue.'''